## **Instalamos pyspark**

In [ ]:
! pip install pyspark

## **Llamamos a su libreria y la guardamos en una variable para usarla**

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

##**Importamos los datos y los procesamos**

In [ ]:
def Leer_texto(pNombreArchivo):
  #creamos un arreglo de tuplas 
  Tuplas=[]
  #procesamos nuestro archivo lo volvemos RDD
  RDD = sc.textFile(pNombreArchivo,100)
  #Iniciamos el numero de twitt
  i=1
  for Linea in RDD.collect():
    #eliminamos los twitt que sean vacios
    if len(Linea) != 0: 
      #Creamos los valores para agregar a la tupla con el valor del numero 
      #de twitt y el twitt
      Tup=(i,Linea)
      #Agregamos la tupla a nuestro arreglo tuplas
      Tuplas.append(Tup)
      #incvremwentamos el valor del twitt
      i+=1
  #devolvemos el arreglo de tuplas
  return Tuplas

In [ ]:
#Procesamos nuestros datos
data = Leer_texto('twitts.txt')

## **Implementar en un notebook pyspark usando mapreduce el algoritmo TF-IDF**

## **Hallamos el TF**

1.   **linesRDD=sc.parallelize(data)**

Creamos nuestro archivo RDD para realizar los procesos del algoritmos TF

2.   **flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])**

Primero lo que hacemos es convertir cada palabra de cada twitt en tokens.
Para esto le pasamos la funcion flatMap el cual junta todo los datos en una sola lista.

Para ello le pasamos la funcion lambda, con este agregamos una tupla la cual contiene una tupla en la posicion 0 y un valor entero en la posicion 1.
La tupla creada almacena el numero del twitt al cual pertecene junto con el token almacenado.

Mientras que el valor entero es 1 el cual representa el numero de veces que se repite inicialmente.

3.   **reduceByKey(lambda x,y:x+y)**

Al observar que existen palabras que se repiten varias veces en un solo twitt pasamos a sumarlos para saber asi el numero exacto de veces que aparece en este mismo.

Para eso usamos reduceByKey el cual nos permite hacer operaciones entre elementos que presentan el mismo Key, esto permitiendonos sumar los valor que se repiten en el mismo twitt y y con el mismo token.

Pasamos la funcion (lambda x,y:x+y) para que sume el valor del value de dicho Key.

4.   **map(lambda x: (x[0][1],(x[0][0],x[1])))**

Con este map reordenamos nuestro RDD para asi presentarlo de una mejor forma.

Primero mostraremos el valor del token.

Segundo tenemos una tupla la cual en su posicion 0 contiene el numero de twitt al cual pertenece, mientras que en su posicion 1 ubicamos el numero de veces que se repite el token dentro del twitt.

5.   **.collect()**

Nos permite mostrar nuestro RDD.

In [ ]:
linesRDD=sc.parallelize(data)
#
tfreducidoRDD =linesRDD.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()]).reduceByKey(lambda x,y:x+y)
#tf ordenado que muestra el token, numero de twitt, repeticion 
tfRDD = tfreducidoRDD.map(lambda x: (x[0][1],(x[0][0],x[1])))
tfRDD.collect()

## **Hallamos el IDF**

1.   **import math**

Importamos la libreria math ya que utilizaremos la funcion log

2.   ** from pyspark.sql.functions import *  **

importamos las functions de pySpark ya que esta nos permitira usar el reduce 

3.   **.map(lambda x: (x[0][1],(x[0][0],x[1],1)))**

Con este map agregaremos un nuveo valor 1 el cual representara la concurrencia del token en todo el corpus

4.   **.map(lambda x:(x[0],x[1][2]))**

Con este map haremos que solo recupere el valor del token y su concurrencia en todo el corpus.

5.   **.reduceByKey(lambda x,y:x+y)**

Con reduceByKey haremos que se sumen los valores de cada token para asi saber su concurrencia en todo el corpus.

6.   **.map(lambda x: (x[0],math.log10(len(data)/x[1])))**

Por ultimo con este map pasaremos la funcon math.log10 para poder hallarlo del tamaño total del corpus entre el numero de veces que aparece un token en este.

In [ ]:
import math
from pyspark.sql.functions import *

In [ ]:
idfRDD=tfreducidoRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1))).map(lambda x:(x[0],x[1][2])).reduceByKey(lambda x,y:x+y).map(lambda x: (x[0],math.log10(len(data)/x[1])))
idfRDD.collect()

[('update', 1.3937460844372076),
 ('aapl', 0.5944950710074257),
 ('stockaction', 2.0249643045063834),
 ('shit', 1.6891722025831901),
 ('evidence', 2.7361687652594138),
 ('suicide', 3.2802368096096894),
 ('store', 1.105595616949241),
 ('turkey', 3.2802368096096894),
 ('business', 2.4351387695954325),
 ('investwall', 3.5812668052736707),
 ('survey', 2.882296800937652),
 ('feedback', 3.104145550554008),
 ('iwatch', 2.627024295834346),
 ('accessory', 2.979206813945708),
 ('gadget', 3.104145550554008),
 ('stage', 1.3798696809532192),
 ('protest', 1.377146822617746),
 ('nyc', 1.353380100659997),
 ('anger', 1.388142206919209),
 ('misplace', 1.390935107103379),
 ('retweet', 1.399423217328898),
 ('try', 1.9792068139457082),
 ('turn', 2.1662934573028525),
 ('speak', 2.627024295834346),
 ('new', 1.3259943001703645),
 ('run', 2.5398741201154458),
 ('tl_chat', 2.979206813945708),
 ('thanks', 2.678176818281727),
 ('call', 2.3025132043208414),
 ('facetime', 2.4351387695954325),
 ('seamlessly', 3.2802

## **Hallamos TF-IDF**

1.   **.join(idfRDD)**

Lo que hacemos es unir el TF con e IDF por medio de los keys lo cual hara una combinacion total entregando asi los valores de Frecuencia y de el Log10 para asi operar con este

2.   **.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))**

Con este map lo que hacemos es ordenar nuestro datos pero ubicando al final la multiplicacion de la frecuencia de un token * el valor de la IDF

3.   **.sortByKey()**

Esto nos permitira realizar la multiplicacion respectiva al numero de twitt que pertenece.

4.   **.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))**

Con este map Mostraremos nuestros datos para su mayo entendimiento.
Primero teniendo el numero de twitt
Segundo el valor del token
Tercero el valor de frecuencia de dicho token
Cuarto el valor del IDF 
Quinto el valor de la frecuencia * idf 


In [ ]:
print("Twitt ","Token ", "TF ", "     IDF     ","     TF-IDF     ")
tfidfrdd=tfRDD.join(idfRDD).map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey().map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
tfidfrdd.collect()

Twitt  Token  TF       IDF           TF-IDF     


[(1, 'store', 1, 1.105595616949241, 1.105595616949241),
 (1, 'stage', 1, 1.3798696809532192, 1.3798696809532192),
 (1, 'nyc', 1, 1.353380100659997, 1.353380100659997),
 (1, 'protester', 1, 1.388142206919209, 1.388142206919209),
 (1, 'diein', 1, 1.399423217328898, 1.399423217328898),
 (1, 'protest', 1, 1.377146822617746, 1.377146822617746),
 (1, 'anger', 1, 1.388142206919209, 1.388142206919209),
 (1, 'misplace', 1, 1.390935107103379, 1.390935107103379),
 (1, 'retweet', 1, 1.399423217328898, 1.399423217328898),
 (1, 'agree', 1, 1.3690792008697128, 1.3690792008697128),
 (2, 'aapl', 1, 0.5944950710074257, 0.5944950710074257),
 (2, 'stockaction', 1, 2.0249643045063834, 2.0249643045063834),
 (2, 'mkt', 1, 1.968482948553935, 1.968482948553935),
 (2, 'range', 1, 2.2195389692560776, 2.2195389692560776),
 (2, 'user', 1, 1.8178388117107334, 1.8178388117107334),
 (2, 'trading', 1, 1.928054291498327, 1.928054291498327),
 (2, 'stock', 1, 1.457415164306585, 1.457415164306585),
 (2, 'update', 1, 1.393